In [2]:
pip install pysrt

     -------------------------------------- 104.4/104.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 199.1/199.1 kB 431.2 kB/s eta 0:00:00
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13444 sha256=7eecc0dcfc780e84a6f6b0aae896c68602eda278ee6408b73c24f3722be6c68c
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\30\a6\ab\4705174e11f44e74d58c14b32edbacbc852644f86658316aef
Successfully built pysrt
Note: you may need to restart the kernel to use updated packages.


In [114]:
import pysrt
import os
import chardet
from pathlib import Path
import difflib
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [101]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [102]:
def extract_text_from_srt(file_path, encoding):
    subs = pysrt.open(file_path, encoding=encoding)
    text = ""
    for sub in subs:
        text += sub.text_without_tags + " "
    return text

In [103]:
def preprocess_text(text):
    HTML = r'<.*?>'
    TAG = r'{.*?}'
    COMMENTS = r'[\(\[][A-Z ]+[\)\]]'
    LETTERS = r'[^a-zA-Z\'.,!? ]'
    SPACES = r' +'
    DOTS = r'[\.]+'

    text = re.sub(HTML, ' ', text) # Заменяем HTML тэги на пробел
    text = re.sub(TAG, ' ', text) # Заменяем тэги на пробел
    text = re.sub(COMMENTS, ' ', text) # Заменяем комменты на пробел
    text = re.sub(LETTERS, ' ', text) # Заменяем все, что не буквы, на пробел
    text = re.sub(SPACES, ' ', text) # Заменяем повторяющиеся пробелы на один пробел
    text = re.sub(DOTS, '.', text)  # Заменяем многоточие на точку
    text = text.encode('ascii', 'ignore').decode() # Удаляем все, что не ASCII символы
    text = ".".join(text.lower().split('.')[1:-1]) # Удаляем первый и последний субтитр (обычно это реклама)
    
    return text

In [125]:
subtitles_directory = r'C:\Users\HP\Documents\python\engsubs\English_scores\Subtitles_all\Subtitles'

In [126]:
output_directory = r'C:\Users\HP\Documents\python\engsubs\Subtitles subtitles'

In [127]:
# Проверяем и создаем папку, если она не существует
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [128]:
# Обход всех файлов субтитров в заданном каталоге
for file_path in Path(subtitles_directory).glob('*.srt'):
    filename = file_path.name

In [129]:
# Обход всех файлов субтитров в заданном каталоге
for file_path in Path(subtitles_directory).glob('*.srt'):
    filename = file_path.name
    
    # Определяем кодировку файла
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    
    # Извлекаем текст из файла субтитров
    extracted_text = extract_text_from_srt(file_path, encoding)
    
    # Предобработка текста
    processed_text = preprocess_text(extracted_text)
    
    # Формируем путь для сохранения текстового файла
    output_file_path = os.path.join(output_directory, filename[:-4] + ".txt")
    
    # Сохраняем предобработанный текст в файл
    with open(output_file_path, 'w', encoding=encoding) as output_file:
        output_file.write(processed_text)